# Exploring data interoperability with STAC and the Microsoft Planetary Computer

By [Pete Gadomski](https://github.com/gadomski).

## Discovery

First, let's find out what collections are available to us on the Planetary Computer.
We'll use [pystac-client](https://pystac-client.readthedocs.io/en/stable/) to query the Planetary Computer's [STAC API](https://github.com/radiantearth/stac-api-spec).

In [ ]:
from pystac_client import Client
from rich.table import Table

PLANETARY_COMPUTER = "https://planetarycomputer.microsoft.com/api/stac/v1"
client = Client.open(PLANETARY_COMPUTER)
collections = list(client.get_all_collections())
collections.sort(key=lambda c: c.id)
table = Table("ID", "Title", title="Planetary Computer collections")
for collection in collections:
    table.add_row(collection.id, collection.title)
table

## Getting our Landsat collection id

As you can see, there's a ton of collections.
Because this is PECORA, let's narrow in on the [Landsat](https://landsat.gsfc.nasa.gov/) collections for now.

In [ ]:
from rich.markdown import Markdown

landsat_collections = [c for c in collections if c.id.startswith("landsat")]
table = Table("ID", "Title", "Description",
              title="Planetary Computer Landsat collections")
for collection in landsat_collections:
    table.add_row(collection.id,
                  collection.title,
                  Markdown(collection.description))
table

## Choosing the correct Landsat collection

As you can see, there's currently three Landsat collections in the Planetary Computer.
Which one to use?
The two `landsat-c2-*` collections seem to be of a pair, but what's going on with `landsat-8-c2-l2`?

Turns out that `landsat-8-c2-l2` is an old collection that has been superseded by `landsat-c2-l2`.
There's no way to know this from the STAC metadata, though we (being the Planetary Computer team) should probably fix that by marking it deprecated with the [version extension](https://github.com/stac-extensions/version).
The deprecated collection is hidden from the website interface, if you use that.

But, for now, we'll use `landsat-c2-l2` for Landsat Level 2 data.

# Fetching and displaying data

First, let's demonstrate loading data using **pystac-client** and [odc-stac](https://github.com/opendatacube/odc-stac).
**odc-stac** is part of the [OpenDataCube](https://www.opendatacube.org/), ecosystem, but stands alone as its own software package.
If you're familiar with OpenDataCube, you know that it's primary product is a database-cum-analysis software stack used for analyzing geospatial data, not dissimilar to the Planetary Computer.
**odc-stac** represents a coming together between the OpenDataCube and the STAC ecosystems, because it take STAC items and turns them into analysis-ready xarrays using routines developed for the OpenDataCube.
**odc-stac** *does not* require a database, or any significant dependencies.

We'll define an area around Steamboat Springs, Colorado, and a datetime range during the winter.
We'll also search for items that have a sufficiently low amount enough of cloud cover to create a good mosaic.
First, let's find the STAC items that match our criteria.

In [ ]:
LANDSAT_COLLECTION = "landsat-c2-l2"
# this is a simplified datetime syntax that pystac-client understands
DATETIME = "2022-06"
CLOUD_COVER_THRESHOLD = 10  # percent
BBOX = [-107.381493, 40.118423, -106.331366, 40.960106]  # Steamboat Springs, CO

item_search = client.search(
    collections=[LANDSAT_COLLECTION],
    bbox=BBOX,
    datetime=DATETIME,
    query=[f"eo:cloud_cover<={CLOUD_COVER_THRESHOLD}"],
)
item_collection = item_search.item_collection()
print(f"Found {len(item_collection)} items")

## Showing the STAC footprints

So far, we've only fetched STAC metadata, which are just JSON objects.
We haven't fetched any raster data at all.
How do the footprints of our STAC items relate to our bounding box of interest?
Let's look, using **folium**.

In [ ]:
from folium import Map, GeoJson
from shapely.geometry import box

item_style_function = lambda s: {
    "color": "#8da0cbaa"
}
bbox_style_function = lambda s: {
    "color": "#fc8d62"
}

map = Map(tiles="OpenStreetMap")
minx = 180
maxx = -180
miny = 90
maxy = -90
for item in item_collection:
    if item.bbox[0] < minx:
        minx = item.bbox[0]
    if item.bbox[1] < miny:
        miny = item.bbox[1]
    if item.bbox[2] > maxx:
        maxx = item.bbox[2]
    if item.bbox[3] > maxy:
        maxy = item.bbox[3]
    GeoJson(item.to_dict(), style_function=item_style_function).add_to(map)
GeoJson(box(*BBOX), style_function=bbox_style_function).add_to(map)
map.fit_bounds([[miny, minx], [maxy, maxx]])
display(map)

## Reading data only in our area of interest

As you can see, the item bounds extend well beyond our bounding box of interest.
While we could load all the data from all items, that's inefficient.
**odc-stac** can use our bounding box and the fact that our Landsat data is stored as Cloud-Optimized GeoTIFFs (COGs) to only read the bits we need, not all of the data.
Let's show that in action, loading only the RGB bands for quick visualization.
Note that we use the Planetary Computer API to sign the asset hrefs, which appends a Shared Access Signature (SAS) to allow access to the data assets.

We load the data at 100m resolution to speed up display operations.
For analysis, you'll most likely want to load at native resolution.

In [ ]:
import odc.stac
import planetary_computer

item_collection = planetary_computer.sign_item_collection(item_collection)
data = odc.stac.load(
    item_collection,
    bands=["red", "green", "blue"],
    groupby="solar_day",
    bbox=BBOX,
    resolution=100
)
data

## Plotting RGB data

There's nothing better than visualizing your data as full-color images.

In [ ]:
rgb_data = data.odc.to_rgba(vmin=1, vmax=15000, bands=["red", "green", "blue"])
rgb_data.plot.imshow(col="time", rgb="band", robust=True)

## Monthly composite using dask

This is all pretty easy work, and works moderatly well when running in a synchronous execution environment.
However, when trying to scale up this type of work into large spatial and temporal domains, some problems are easily parallelizable.
[Dask](https://www.dask.org/) is a powerful parallel execution enviornment that works well in single-user, large data environments, such as geospatial analysis.
The Planetary Computer Hub includes a Dask Gateway cluster for use for analysis; you can also set up Dask Gateway using [local processes](https://gateway.dask.org/install-local.html), e.g. on your laptop.

First, let's set up a gateway cluster.
You can click on the dashboard link to see the execution status of the Dask cluster.

In [ ]:
#cluster_type = 'Coiled'
#cluster_type = 'Gateway'
#cluster_type = 'Local'
cluster_type = 'Local'

In [ ]:
if cluster_type == 'Coiled':
    import coiled
    cluster = coiled.Cluster(
        region="us-west-2",
        arm=True,   # run on ARM to save energy & cost
        worker_vm_types=["t4g.small"],  # cheap, small ARM instances, 2cpus, 2GB RAM
        worker_options={'nthreads':2},
        n_workers=30,
        wait_for_workers=False,
        compute_purchase_option="spot_with_fallback",
        name='landsat',   # Dask cluster name
        software='protocoast-develop-arm',  # Conda environment name
        workspace='osc-aws',
        timeout=180   # leave cluster running for 3 min in case we want to use it again
    )

    dask_client = cluster.get_client()

In [ ]:
if cluster_type == 'Gateway':
    from dask_gateway import Gateway
    import os
    gateway = Gateway()
  # Set Dask Gateway cluster options 
    options = gateway.cluster_options()
    options.image = os.environ['JUPYTER_IMAGE']   # set worker image same as notebook
    print('Setting Cluster Image',options.image) 
    # Now start a new cluster which has these options
    print('Starting new cluster.')
    cluster = gateway.new_cluster(options)   
    cluster.adapt(minimum=4, maximum=24)
    dask_client = cluster.get_client()

In [ ]:
if cluster_type == 'Local':
    from dask.distributed import LocalCluster
    from dask.distributed import Client as DaskClient
    cluster = LocalCluster()
    dask_client = DaskClient(cluster)

In [ ]:
dask_client

In [ ]:
# cluster.shutdown()

Next, let's find images for the same area of interest, but this time for the whole year of 2021.

In [ ]:
item_search = client.search(
    collections=[LANDSAT_COLLECTION],
    bbox=BBOX,
    datetime="2021",
    query=[f"eo:cloud_cover<=25"],
)
item_collection = planetary_computer.sign_item_collection(item_search.item_collection())
print(f"Found {len(item_collection)} items")

By using the `chunks` option to `odc.stac.load`, we instruct **odc-stac** to break up the data loading into a set of tasks, which will be fanned out to Dask workers on request.
Note that we haven't actually loaded any data yet; we've only built the execution graph, or plan.
Your Dask gateway status page should be still.

In [ ]:
data = odc.stac.load(
    item_collection,
    bands=["red", "green", "blue"],
    bbox=BBOX,
    resolution=100,
    groupby="solar_day",
    chunks={"x": 2048, "y": 2048},
)
data

By asking our xarray to persist its data, we now actually load the data into our xarray on our workers.

In [ ]:
data.persist()

We can create monthly composites of the band data, which will be done on the Dask cluster as well.

In [ ]:
%%time
monthly_data = data.groupby("time.month").median().compute()
monthly_data

Finally, let's display!
We could do a lot more to create better composities, but we'll leave that as an exercise for the reader.

In [ ]:
monthly_data_rgb = monthly_data.odc.to_rgba(vmin=1, vmax=15000, bands=["red", "green", "blue"])
monthly_data_rgb.plot.imshow(x="x", y="y", rgb="band", col="month", col_wrap=3, figsize=(6, 8))